# Elasticsearch use case

Here we try the possibility given by elasticsearch and its python high level sdk, elasticsearch_dsl.

First, import the needed modules and create a connection.

Here we assume an istance of elasticsearch is running on localhost, on the default port.

In [ ]:
from datetime import datetime

from elasticsearch_dsl import (
    DocType, Keyword, Date, Float, Nested, InnerObjectWrapper, Search
)
from elasticsearch_dsl.aggs import A
from elasticsearch_dsl.connections import connections
from elasticsearch_dsl.query import Q


# Connect to a local ES instance
connections.create_connection(hosts=['localhost'], timeout=20)


## Define the doc type

Next we define a DocType, BasicMetric, that will tell ES how to represent our objects.

If this is not set first, ES will create it by itself the first time you send data to it.

We create it manually so we can define some fields as Keyword rather than Text where we 
won't need Full Text Search. This will make the index faster, and will avoid the creation
of a wrong index caused by malformed data

In [ ]:
class BasicMetric(DocType):
    """
    Besic metric mapping
    """
    metric_type = Keyword()
    user_id = Keyword()
    timestamp = Date()
    value = Float()
    metadata = Nested(
        properties={
            'type': Keyword(),
            'unique_id': Keyword(),
        }
    )

    class Meta:
        index = 'metrics'


## Initialize metric

We can now add one record by istantiating a BasicMetric object and calling the save() method (much like the Django ORM)

In [ ]:
metric = BasicMetric(metric_type='time', user_id='prova id', timestamp=datetime.now(), value=1.245)
metric.metadata.append({'type': 'provatype', 'unique_id': 'very unique id'})
metric.save()
print("==> Metric: %s" % metric)


## An example search

Here is how we can make call to ES. Let's say we want to ask for all our records, limiting the fields that will be returned:

In [ ]:
s = Search()
s = s.source(['user_id', 'value', 'metadata'])
s.to_dict()
response = s.execute()
print("==> Success:\t%s" % response.success())
print("==> Total hits:\t%s" % response.hits.total)

## Some usefull search queries

Elasticsearch DSL allow us to build queries with a DjangoORM-like syntax.
Search objects have methods like .filter() and .exclude().

We can build aggregations using the A object and we can build queries using Q objects
and combine them with logic operators.



### Get all records with a time metric value between 2000 and 30000


In [ ]:
s = Search().filter('range', value={"gte": 2000, "lte": 30000})
response = s.execute()
print("==> Response:")
print(response)
print("==> Hits:")
print(response.hits[0])
print("==> First user id:")
print(response.hits[0].user_id)

### Number of records per user

This is how we can build an aggregation.
We can also use the to_dict() method to show the query we just built, and if we set the size of the Search object to 0, we can retrieve only the aggregations.

In [ ]:
# Define aggregation
agg = A('terms', field='user_id', include='user_id')
# init search
s = Search()
# Size 0
s = s[0:0]
# Add aggregation
s.aggs.bucket('group_by_user', agg)
# Dict representation
s.to_dict()

### Average time by segment

Here we use the Q object to build the query

In [ ]:
s = Search()

# Set size to 0, we only want aggregations
s = s[0:0]

# Queries
s = s.filter('term', metric_type='time')
q = Q('term', metadata__resource_type='segment')
s = s.filter('nested', path='metadata', query=q)

# Aggregations (read it down-top)
a = A('avg', field='value')
a = A('reverse_nested', aggs={'avg_time': a})
a = A('terms', field='metadata.resource_id', aggs={'average': a})
a = A('nested', path='metadata', aggs={'segments': a})
s.aggs.bucket('by_segment_id', a)

s.to_dict()

resp = s.execute()
print("==> Buckets: %s" % resp.aggregations.by_segment_id.segments.buckets)
print("==> Doc count: %s" % resp.aggregations.by_segment_id.segments.buckets[0].doc_count)


### Best time for a given segment

In [ ]:
s = Search()

# We only want the first result and user_id and value fields
s = s[0:1]
s = s.source(['user_id', 'value'])

# Sorting
s = s.sort('-value')

# Filter it
s = s.filter('term', metric_type='time')
q = Q('term', metadata__resource_type='segment')
q &= Q('term', metadata__resource_id='segment-id')
s = s.filter('nested', path='metadata', query=q)

s.to_dict()

In [ ]:
s.execute() # No results if we don't set an existing segment_id

### Best time in a given segment for a given user

In [ ]:
q1 = {'term': {"metadata__resource_type": 'segment'}}
q2 = {'term': {"metadata__resource_id": 'segment-id'}}
q = Q('bool', filter=[q1,q2])

s = Search()[0:1]\
        .source(['user_id', 'value'])\
        .sort('-value')\
        .filter('term', metric_type='time')\
        .filter('term', user_id='user-id')\
        .filter('nested', path='metadata', query=q)

s.to_dict()